In [1]:

import time

from ibvpy.bcond import BCSlice
from ibvpy.fets import FETS2D4Q
from ibvpy.tmodel.mats2D.mats2D_elastic import MATS2DElastic
from ibvpy.tmodel.mats3D.mats3D_plastic.vmats3D_desmorat import \
    MATS3DDesmorat
from ibvpy.tmodel.viz3d_tensor_field import \
    Vis3DTensorField, Viz3DTensorField
from mayavi import mlab
from ibvpy.api import \
    TStepBC
from ibvpy.xmodel.xdomain_fe_grid import XDomainFEGrid

import numpy as np

from notebooks.sim_examples.mlab_decorators import decorate_figure

In [2]:
thickness = 1

xdomain = XDomainFEGrid(coord_max=(1, 1),
                        shape=(1, 1),
                        integ_factor=thickness,
                        fets=FETS2D4Q())

In [3]:
tmodel = MATS3DDesmorat()
tmodel = MATS2DElastic(E=1, nu=0)

left_y = BCSlice(slice=xdomain.mesh[0, :, 0, :],
                 var='u', dims=[1], value=0)
left_x = BCSlice(slice=xdomain.mesh[0, :, 0, :],
                 var='u', dims=[0], value=-1)
right_x = BCSlice(slice=xdomain.mesh[-1, :, -1, :],
                  var='u', dims=[0], value=0.0)

In [4]:
m = TStepBC(
    domains=[(xdomain, tmodel)],
    bc=[left_x, right_x, left_y],
)

In [5]:
m.hist.vis_record = {
    'strain': Vis3DTensorField(var='eps_ab'),
    'stress': Vis3DTensorField(var='sig_ab'),
    #        'damage': Vis3DStateField(var='omega_a'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}


In [6]:
s = m.sim
s.tloop.k_max = 1000
s.tline.step = 0.1
s.tloop.verbose = True
s.run()

t:	 0.00(0), 
	 0.10(1), 
	 0.20(1), 
	 0.30(1), 
	 0.40(1), 
	 0.50(1), 
	 0.60(1), 
	 0.70(1), 
	 0.80(1), 
	 0.90(1), 
	 1.00(1), 



In [7]:
print('area', thickness)
F_ti = s.hist.F_t
print('left')
print(np.sum(F_ti[-1, right_x.dofs]))
print('right')
print(np.sum(F_ti[-1, left_x.dofs]))

mlab.options.backend = 'envisage'

f_strain = mlab.figure()
scene = mlab.get_engine().scenes[-1]
scene.name = 'strain'
strain_viz = Viz3DTensorField(vis3d=s.hist['strain'])
strain_viz.setup()

decorate_figure(f_strain, strain_viz, 200, [70, 20, 0])

#mlab.show()

area 1
left
0.9999999999999996
right
-0.9999999999999996
